In [2]:
import glob
import pandas as pd
import nltk
from nltk import word_tokenize, sent_tokenize
# from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron


condition data

In [3]:
def conditioner(text_column):
    text_column = text_column.astype(str) #make sure type string
    text_column=text_column.apply(lambda x: " ".join(x.lower() for x in x.split() ))#make lowercase
    text_column=text_column.str.replace('[^\w\s]','') #remove punctuation
    #remove stop words borred from library
    stop= pd.read_csv("english_stop.txt")
    text_column=text_column.apply(lambda x: " ".join(x for x in x.split() if x not in stop ))
    #stemming
    st = PorterStemmer()
    text_column=text_column.apply(lambda x: " ".join([st.stem(word) for word in x.split()] ))
    return  text_column

import files to be trained, we will 6,000 reviews total: 3,000 + and 3,000 -

In [6]:
path2 = r'data' # use your path
all_files2 = glob.glob(path2 + "/*.csv")

sentiment_dat = []

for i in range(3):
    
    df = pd.read_csv(all_files2[i], index_col=0, header=0)
    df.insert(0, 'company', all_files2[i][:-4].replace("data", "").strip('/').replace("-"," ").strip("\\"))
    #df['company'] = str(filename).strip('.csv').strip('data/')
    sentiment_dat.append(df)

frame2 = pd.concat(sentiment_dat, axis=0, ignore_index=True)

conditioning the data
All the Pros get assigned to positive review of label +1, all the cons to negative review of label -1

In [11]:
positive = pd.DataFrame(frame2["pros"].values)
negative = pd.DataFrame(frame2["cons"].values)
positive = positive.rename(columns = {"pros": "text"})
negative = negative.rename(columns = {"cons": "text"})
positive['sentiment'] = 1
negative['sentiment'] = -1 #changed to -1 to be used later in clustering
positive = positive.replace(r'\n','', regex=True) 
negative = negative.replace(r'\n','', regex=True) 
# labeling has been decided  depending of cons or pros
frames=[positive, negative]
reviews=pd.concat(frames, axis=0, ignore_index=False)
all_reviews = pd.concat(frames, axis=0, ignore_index=False)
all_reviews=all_reviews.rename(columns = {0: "text"})
##
#Apply conditioner function
##
all_reviews['text']=conditioner(all_reviews['text'])
reviews = (all_reviews['text'].values)
labels = (all_reviews['sentiment'].values)

In [24]:
all_reviews

,text,sentiment
0,interest work at the compani,1
1,good pay benefit and worklif balanc a lot of o...,1
2,valu your idea on a team,1
3,good benefist on thi compani,1
4,money money money money money,1
...,...,...
3003,insan attent to planningpoor processestough wo...,-1
3004,career growth is tough and some thing are not ...,-1
3005,insular cultur silo head by empir builder posi...,-1
3006,not savvi in the use of it as much as it could...,-1


preparation for training

In [ ]:
from sklearn.model_selection import train_test_split

In [13]:
text_train, text_test, sentiment_train, sentiment_test = train_test_split(reviews, labels, test_size=0.2, random_state=45)
text_train,text_val, sentiment_train, sentiment_val = train_test_split(text_train, sentiment_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

from sklearn.pipeline import Pipeline
vectorizer=CountVectorizer()
clf= LogisticRegression()
vec_clf = Pipeline([('countV', vectorizer), ('svm', clf)])
vec_clf.fit(text_train,sentiment_train)



Pipeline(steps=[('countV', CountVectorizer()), ('svm', LogisticRegression())])

In [18]:
accuracy_val= vec_clf.score(text_val, sentiment_val)

In [19]:
print("Accuracy validation of LR:", accuracy)

Accuracy validation of LR: 0.9276807980049875


In [20]:
accuracy= vec_clf.score(text_test, sentiment_test)

In [21]:
print("Accuracy of LR:", accuracy)

Accuracy of LR: 0.9260797342192691


comparing model with vader

In [22]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA


In [23]:
nltk.download('vader_lexicon') #pretrained Penn State

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\kikik\AppData\Roaming\nltk_data...


True

In [25]:
abstracts=all_reviews['text']

In [26]:
sia = SIA()
results = []

for line in abstracts:#headlines:
    pol_score = sia.polarity_scores(line)
    pol_score['text'] = line
    results.append(pol_score)


In [29]:
df = pd.DataFrame.from_records(results)
df.head(2)

,neg,neu,pos,compound,text
0,0.000,0.571,0.429,0.4588,interest work at the compani
1,0.073,0.622,0.306,0.6705,good pay benefit and worklif balanc a lot of o...


In [31]:
#setting treshhold

df['sentiment'] = 0
df.loc[df['compound'] > 0, 'sentiment'] = 1  
df.loc[df['compound'] < -0, 'sentiment'] = -1
df.loc[df['compound'] ==0, 'sentiment'] = -1  #no neutrals, rescale
df.head()

,neg,neu,pos,compound,text,label,sentiment
0,0.000,0.571,0.429,0.4588,interest work at the compani,1,1
1,0.073,0.622,0.306,0.6705,good pay benefit and worklif balanc a lot of o...,1,1
2,0.000,1.000,0.000,0.0000,valu your idea on a team,-1,-1
3,0.000,0.580,0.420,0.4404,good benefist on thi compani,1,1
4,0.000,1.000,0.000,0.0000,money money money money money,-1,-1


In [35]:
all_reviews2 = df.filter(['text','sentiment'], axis=1)

In [ ]:
reviews2 = (all_reviews2['text'].values)
labels2 = (all_reviews2['sentiment'].values)

now lets compare accuracy of valder with the test data

getting the prediction for our text_test data

In [38]:
sia = SIA()
results = []

for line in text_test:#headlines:
    pol_score = sia.polarity_scores(line)
    pol_score['text'] = line
    results.append(pol_score)

In [40]:
df = pd.DataFrame.from_records(results)
df.head(2)

,neg,neu,pos,compound,text
0,0.0,0.413,0.587,0.7964,great benefit peopl orient opportun for advanc
1,0.0,0.594,0.406,0.6249,great learn opportun for earli career stage


In [41]:
#setting treshhold

df['sentiment'] = 0
df.loc[df['compound'] > 0, 'sentiment'] = 1  
df.loc[df['compound'] < -0, 'sentiment'] = -1
df.loc[df['compound'] ==0, 'sentiment'] = -1  #no neutrals, rescale
df.head()

,neg,neu,pos,compound,text,sentiment
0,0.0,0.413,0.587,0.7964,great benefit peopl orient opportun for advanc,1
1,0.0,0.594,0.406,0.6249,great learn opportun for earli career stage,1
2,0.0,1.000,0.000,0.0000,larg and global with divers opportun,-1
3,0.0,0.459,0.541,0.7096,benefit retir 401k good work life balanc,1
4,0.0,1.000,0.000,0.0000,cheap snack and drink tv when the bill is paid...,-1


In [47]:
number_oftrue=(df['sentiment']==sentiment_test).astype(int)

In [49]:
number_oftrue.sum()

852

In [50]:
number_oftrue.sum()/len(sentiment_test)

0.707641196013289

accuracy = correct predictions/all predictions